#  Importación de librerías

- Se importan los módulos de torch para redes neuronales y entrenamiento
- dataset se usa para cargar conjunto de datos
- transformes facilita el uso de modelos NLP
- skelearn.metrics se usa para evaluar el rendimiento del modelo

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors
from transformers import PreTrainedTokenizerFast

## Carga del conjunto de datos

- AG_NEWS es un dataset de clasificación de texto
- se prepara el entrenamiento (dataset_train) y prueba (dataset_test)

In [5]:
# Load AG_NEWS dataset using the datasets library
dataset = load_dataset("ag_news")
dataset_train = dataset["train"]
dataset_test = dataset["test"]

In [6]:
len(dataset_train)

120000

# Creación de un Tokenizador BPE

- Se utiliza tokenizers para definir un tokenizador basado en Byte Pair Encoding (BPE)
- Se configura para dividir texto en palabras usando espacios en blanco

In [ ]:
# Initialize a BPE tokenizer
tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
# Set the pre-tokenizer to split on whitespace
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Entrenamiento del tokenizador

- Se define un entrenador BPE con un vocabulario de 1000 tokens
- Seagregan tokens especiales como [PAD], [MASK], etc.
- Se entrena el tokenizador en lotes (batch_iterator)

In [ ]:


# Set up the trainer with special tokens
trainer = trainers.BpeTrainer(
    vocab_size=1000,
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

# Define a generator to yield batches of text
def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset_train), batch_size):
        yield dataset_train[i:i + batch_size]["text"]

# Train the tokenizer
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

# Post-Procesamiento y guardado del tokenizador 

- Usa *processors.TemplateProcessing* para definir cómo se manejan las secuencias individuales y pares de secuencias
- Se configuran reglas de procesamiento de texto con tokens [CLS] y [SEP]
- Se habilita truncamiento y padding hasta 128 tokens.
- *tokenizer.enable_padding(...)* asegura que todas las secuencias tengan exactamente 128 tokens, rellenando con [PAD] si es necesario
- Se guarda el tokenizador en un archivo JSON

In [24]:
# Set the post-processor to add special tokens
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)
# Enable truncation and padding
tokenizer.enable_truncation(max_length=128)
tokenizer.enable_padding(pad_id=tokenizer.token_to_id("[PAD]"), pad_token="[PAD]", length=128)

tokenizer.save("custom_tokenizer.json")

# Cagado del tokenizador guardado

- usa *PreTrainedTokenizerFast* para cargar el tokenizador desde el archivo *custom_tekenizer.json*
- Esto permite reutilizar el tokenizador sin necesidad de configurarlo nuevamente

In [9]:
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")

# Definición de la función *preprocess_data*

- Esta función convierte textos en representaciones numéricas para que puedan ser utilizadas por un modelo de ML
- Toma una lista de textos (*example["text"]*) y la tokeniza en batch
- *encode_batch* convierte cada texto en una secuencia de tokens según las reglas del tokenizador
- *inputs_ids* son los IDs de los tokens generados por el tokenizador
- *attention_mask* indica qué posiciones en la secuencia deben ser atendidas por el modelo (1=token, 0=padding)
- Devuelve un diccionario con los *inputs_ids*, *attention_mask* y las etiquetas originales *(examples["label])*
- Esto permite que los datos sean utilizados directamente en el entrenamiento de un modelo. 

In [10]:
def preprocess_data(examples):
    # Tokenize the text
    encodings = tokenizer.encode_batch(examples["text"])

    # Extract input IDs and attention masks
    input_ids = [encoding.ids for encoding in encodings]
    attention_masks = [encoding.attention_mask for encoding in encodings]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "label": examples["label"]
    }


# Aplicación de la función *preprocess_data*

- Se aplica la función al conjunto de datos de entrenamiento (*dataset_train*) y prueba (*dataset_test*)
- Usa *map* para aplicar *preprocess_data* a cada ejemplo en *dataset_train* y *dataset_test*
- *batched=True* significa que la función recibe múltiples ejemplos a la vez, lo que mejora la eficiencia.
- L función *preprocess_data* convierte el texto en *input_ids* y *attention_mask*
*map* almacena los resultados en un nuevo conjunto de datos (*ds_train* y *ds_test*)

In [ ]:
# Apply preprocessing
ds_train = dataset_train.map(preprocess_data, batched=True)
ds_test = dataset_test.map(preprocess_data, batched=True)

- Este diccionarioo muestra como se transformó el texto original en una representación numérica para el modelo. 
- Elementos clave:
    - *text*: Contiene la frase original.
    - *label*: La categoría asignada al texto (en este caso, 2).
    - *input_ids*:Es la versión tokenizada del texto, donde cada palabra (o subpalabra) ha sido convertida en un número entero. El primer número 1 representa el token [CLS], que indica el inicio de la secuencia. Los ceros al final son padding para alcanzar la longitud máxima (128).
    - *attention_mask*: Indica qué tokens son relevantes (1) y cuáles son relleno (0). Los primeros valores 1 corresponden a los tokens reales, mientras que los ceros al final ignoran el padding.

In [12]:
ds_train[0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2,
 'input_ids': [1,
  54,
  150,
  202,
  16,
  33,
  996,
  34,
  699,
  33,
  215,
  157,
  99,
  95,
  923,
  215,
  11,
  219,
  12,
  219,
  15,
  383,
  692,
  15,
  78,
  245,
  133,
  14,
  54,
  150,
  202,
  104,
  124,
  10,
  78,
  63,
  82,
  286,
  71,
  101,
  58,
  61,
  112,
  103,
  745,
  144,
  15,
  62,
  84,
  73,
  605,
  14,
  192,
  151,
  64,
  101,
  66,
  104,
  96,
  362,
  16,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

# Modelo básico de embedding para claseificar textos

- Es una red neuronal simple con una capa de embeddings y una capa lineal
- Definición de la clase *BasicEmbeddingModel*: Hereda de nn.Module, lo que significa que es un modelo de PyTorch.
- *__init__ (Inicialización del modelo)*:
	- *vocab_size*: Tamaño del vocabulario (cantidad de palabras únicas que el modelo puede manejar).
	- *embed_dim*: Dimensión de los embeddings (representación vectorial de cada palabra).
	- *num_classes*: Número de categorías para la clasificación.
	- *self.embedding* = nn.Embedding(vocab_size, embed_dim, padding_idx=3). Capa de embeddings que transforma los input_ids (tokens) en vectores. *padding_idx=3* hace que los tokens de padding no contribuyan al aprendizaje.
	- *self.fc* = nn.Linear(embed_dim, num_classes). Capa lineal que toma el embedding final y lo proyecta al número de clases.
- *forward (Propagación hacia adelante)*:
    - *embedded = self.embedding(input_ids)* Convierte los tokens en vectores de embedding.
	- *embedded = torch.mean(embedded, dim=1)* Promedia los embeddings a lo largo de la dimensión de las palabras (pooling promedio).
	- *logits = self.fc(embedded)* Pasa el embedding resultante por la capa lineal para obtener las predicciones.
- Cálculo de la pérdida (opcional):
	- Se usa *nn.CrossEntropyLoss()* para calcular la pérdida entre logits y las etiquetas reales.
	- Se devuelve solo la salida (logits), que contiene las predicciones del modelo.



In [13]:
# Define Basic Embedding Model
class BasicEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        embedded = torch.mean(embedded, dim=1)  # Averaging embeddings
        logits = self.fc(embedded)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

# Configuración del modelo de embeddings

- Definir hiperparámetros del modelo
	- *embed_dim = 100*: Cada palabra será representada por un vector de 100 dimensiones.
	- *num_classes = 4*: El modelo clasificará el texto en 4 categorías posibles.
	- *vocab_size = 1000*: El modelo tiene un vocabulario de 1000 palabras únicas.
- Se crea un modelo *BasicEmbeddingModel* con los parámetros anteriores
- Se imprime la arquitectura del modelo.

_Arquitectura del modelo:_

*Embedding(1000, 100, padding_idx=3)*

    - Es la capa de embeddings que toma tokens (números) y los convierte en vectores de tamaño 100.
    - 1000 es el tamaño del vocabulario (cantidad de palabras únicas que el modelo reconoce).
    - *padding_idx=3* significa que el índice 3 corresponde al token de padding, que no contribuirá al entrenamiento.
    
*Linear(in_features=100, out_features=4, bias=True)*

    - Es la capa lineal de clasificación.
    - *in_features=100*: Recibe un vector de 100 dimensiones (el embedding promediado de las palabras).
    - *out_features=4*: Produce 4 valores de salida (uno para cada clase de clasificación).
    - *bias=True*: Incluye un término de sesgo en la capa lineal.

In [14]:
# Model setup
embed_dim = 100
num_classes = 4
vocab_size = 1000
model = BasicEmbeddingModel(vocab_size, embed_dim, num_classes)
# Show model summary
print(model)

BasicEmbeddingModel(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


# Métricas de evaluación para el modelo 

- Recibe eval_pred, que es una tupla (logits, labels), donde:
    - logits: Son las salidas del modelo antes de aplicar softmax.
    - labels: Son las etiquetas reales del conjunto de prueba.
    - np.argmax(logits, axis=1): Toma el índice del valor más alto en cada fila de logits, que corresponde a la clase predicha.
    - accuracy_score(labels, predictions): Compara las predicciones con las etiquetas verdaderas y calcula el porcentaje de aciertos.
- f1_score(labels, predictions, average='weighted'): Calcula el F1-score considerando el equilibrio entre precisión y exhaustividad (precision y recall). La opción 'weighted' pondera cada clase según su frecuencia en el conjunto de datos.

In [15]:
# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

Configuración los argumentos de entrenamiento para el modelo usando la clase TrainingArguments de la biblioteca transformers. Define aspectos clave del proceso de entrenamiento, como el directorio de salida, la frecuencia de evaluación y guardado, la tasa de aprendizaje y el tamaño del lote.

*Tamaño del lote (batch_size):*
- 32 ejemplos por dispositivo durante el entrenamiento.
- 32 ejemplos por dispositivo durante la evaluación.
- Si se usa una GPU, cada GPU procesará un lote de 32 ejemplos.

*Número de épocas:* El modelo pasará tres veces por el conjunto de datos de entrenamiento.


In [16]:

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
)

Este código crea un objeto Trainer, una clase de la biblioteca transformers que automatiza el entrenamiento y evaluación de modelos de deep learning.

- Se crea una instancia de Trainer, que maneja el entrenamiento y evaluación del modelo.
- Modelo a entrenar, en este caso, model, que es una instancia de BasicEmbeddingModel
- Argumentos de entrenamiento, previamente definidos en training_args (como tasa de aprendizaje, número de épocas, etc.).
- Conjuntos de datos: *ds_train* (Datos de entrenamiento) y  *ds_test* (Datos de evaluación).
- Tokenizador: fast_tokenizer es el tokenizador preentrenado que se usará para procesar los datos antes de alimentarlos al modelo.
- Función de métricas: Usa la función compute_metrics, que calcula precisión (accuracy) y puntuación F1 (f1-score) en cada evaluación.


In [17]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_test,
    processing_class=fast_tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
# Train model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.483400,0.483327,0.831711,0.831137
2,0.432300,0.446807,0.843816,0.843414
3,0.425600,0.440676,0.845000,0.844646


TrainOutput(global_step=11250, training_loss=0.5115090135362413, metrics={'train_runtime': 64.1468, 'train_samples_per_second': 5612.125, 'train_steps_per_second': 175.379, 'total_flos': 0.0, 'train_loss': 0.5115090135362413, 'epoch': 3.0})

El entrenamiento del modelo mostró una mejora progresiva en su desempeño a lo largo de tres épocas. La pérdida de entrenamiento disminuyó de 0.4834 en la primera época a 0.4256 en la tercera, indicando que el modelo ajustó mejor los datos con cada iteración. De manera similar, la pérdida de validación se redujo de 0.4833 a 0.4407, lo que sugiere una mejora en la capacidad de generalización del modelo. En términos de precisión, el modelo pasó de un 83.17 % en la primera época a un 84.50 % en la última, mientras que el puntaje F1, que mide el equilibrio entre precisión y sensibilidad, aumentó de 83.11 % a 84.46 %. Estos resultados reflejan un aprendizaje estable y una mejora en la capacidad del modelo para realizar predicciones precisas sin evidencias de sobreajuste.

# Lab: Enhancing a Basic Embedding Model with Positional Encoding and Multi-Head Attention

## Objective
In this lab, you will modify a basic sentiment analysis model by adding two key components from transformer-based architectures: **positional encoding** and **multi-head attention**. These modifications will improve the model’s ability to capture word order and token interactions, making it more expressive.

---

## Task Overview
You are provided with a basic embedding model that performs sentiment analysis by:
- Mapping token IDs to embeddings.
- Averaging embeddings across tokens.
- Passing the result through a fully connected layer for classification.

Your tasks:
1. **Implement positional encoding** to enrich word embeddings with information about token positions.
2. **Replace the mean pooling operation** with a **multi-head self-attention layer** to allow tokens to attend to each other.

---

## Step 1: Implement Positional Encoding
Transformers lack recurrence, so they rely on positional encodings to incorporate token order. You will implement **sinusoidal positional encoding**, as described in Vaswani et al. (2017), and add it to the input embeddings.

### Requirements
- Implement a function to generate sinusoidal positional encodings.
- Ensure the encoding is applied correctly to all input embeddings.

---

## Step 2: Implement Multi-Head Self-Attention
Instead of simply averaging word embeddings, you will replace this operation with **multi-head self-attention** to allow interactions between tokens.

### Requirements
- Implement a multi-head self-attention layer.
- Replace the `torch.mean(embedded, dim=1)` operation with the **attention mechanism**.
- Ensure that the implementation correctly processes padded tokens.

---


## Starter Code
Below is the base implementation of the model. You must complete the missing parts and train the model.

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, num_heads):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        
        # Task 1: Implement positional encoding
        self.positional_encoding = PositionalEncoding(embed_dim)

        # Task 2: Implement Multi-Head Attention (replace mean pooling)
        # self.multihead_attn = ???

        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        
        # Apply positional encoding
        embedded = self.positional_encoding(embedded)

        # Task: Replace mean pooling with Multi-Head Attention
        # attn_output, _ = ???(embedded, embedded, embedded, key_padding_mask=attention_mask)
        
        # Pooling: Extract the first token's representation (CLS token equivalent)
        # pooled_output = attn_output[:, 0, :]

        logits = self.fc(pooled_output)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

# Students must implement this class
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        self.encoding = self.create_positional_encoding(embed_dim, max_len)
        
    def create_positional_encoding(self, embed_dim, max_len):
        # Task: Implement sinusoidal positional encoding here
        pass

    def forward(self, x):
        # Task: Add positional encoding to embeddings
        pass
```

## Evaluation Criteria

To verify your implementation:

- Ensure that positional encoding correctly modifies embeddings.

- Confirm that the self-attention layer replaces mean pooling effectively.

- Train the modified model on a small sentiment dataset and compare results.

## Extra Challenges

For advanced students:

- Experiment with different pooling strategies (e.g., max pooling, CLS token extraction).

- Compare learned positional embeddings vs. sinusoidal encodings.

- Add layer normalization after attention.


In [26]:
import torch
import torch.nn as nn
import math


# Tarea 1: Implementación de Positional Encoding 

Introduce una representación posicional en los embeddings para que el modelo pueda capturar información sobre el orden de las palabras en una secuencia. A diferencia de los modelos tradicionales que dependen de estructuras recurrentes (como RNNs), los modelos basados en atención, como Transformers, no tienen una noción inherente de orden, por lo que se debe agregar manualmente esta información a través de codificaciones posicionales.

1.	Inicialización de la clase
- *PositionalEncoding(nn.Module)*: Se define la clase como una subclase de nn.Module.
- *embed_dim*: Dimensión del espacio de embedding.
- *max_len=5000*: Longitud máxima de la secuencia a considerar.
2.	Creación de la matriz de codificación posicional (create_positional_encoding)
- *position = torch.arange(max_len).unsqueeze(1)*: Crea un tensor con las posiciones de cada token en la secuencia, expandido a una dimensión extra para operaciones posteriores.
*div_term = torch.exp(torch.arange(0, embed_dim, 2) * -(math.log(10000.0) / embed_dim))*: Calcula los factores de escala para las funciones seno y coseno, siguiendo la fórmula estándar de codificación sinusoidal.
- *pe = torch.zeros(max_len, embed_dim)*: Inicializa la matriz de codificación posicional con ceros.
- *pe[:, 0::2] = torch.sin(position * div_term)*: Aplica la función seno a las posiciones de índice par.
- *pe[:, 1::2] = torch.cos(position * div_term)*: Aplica la función coseno a las posiciones de índice impar.
- *return pe.unsqueeze(0)*: Agrega una dimensión para que coincida con la forma (1, max_len, embed_dim) y pueda sumarse fácilmente a los embeddings.
3.	Propagación (forward)
- *x + self.encoding[:, :x.size(1)]*: Se suma la codificación posicional a los embeddings de entrada, asegurando que la secuencia tenga la misma longitud que la entrada.

El propósito de esta implementación es mejorar la capacidad del modelo para distinguir la posición de cada palabra en una oración, lo que es crucial en tareas de procesamiento de lenguaje natural. La combinación de senos y cosenos permite que la representación posicional generalice mejor a longitudes de secuencia que el modelo no ha visto durante el entrenamiento.

In [ ]:

# Task 1: Implement Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        self.encoding = self.create_positional_encoding(embed_dim, max_len)
        
    def create_positional_encoding(self, embed_dim, max_len):
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * -(math.log(10000.0) / embed_dim))
        pe = torch.zeros(max_len, embed_dim)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)  

    def forward(self, x):
        return x + self.encoding[:, :x.size(1)]

# Tarea 2: Implementación de Multi-Head Attention (Reemplazo del Mean Pooling)

Esta tarea introduce la Atención Multi-Cabeza (Multi-Head Attention, MHA) en lugar de la estrategia de Mean Pooling utilizada previamente. La MHA permite que el modelo capture relaciones complejas entre palabras en la secuencia, proporcionando una representación más rica y contextualizada.

*Explicación del código*:

Inicialización del modelo (__init__)
- *embed_dim*: Dimensión del embedding para cada palabra.
- *num_heads*: Número de cabezas de atención en la capa Multi-Head Attention.
- *vocab_size*: Número total de palabras en el vocabulario.
- *max_len=5000*: Longitud máxima de las secuencias para la codificación posicional.

Componentes principales
1. *Embedding Layer (self.embedding).* Convierte los índices de las palabras en vectores densos de embed_dim dimensiones.
2. *Positional Encoding (self.pos_encoding).* Se añade la información de la posición de las palabras en la secuencia, para que el modelo pueda distinguir el orden.
3. *Multi-Head Attention (self.multihead_attn).* Implementa la capa de Atención Multi-Cabeza para capturar relaciones a largo plazo dentro de la secuencia. *nn.MultiheadAttention(embed_dim, num_heads)*, permite que el modelo aprenda diferentes representaciones de atención en paralelo.
4. *Layer Normalization (self.norm)*. (Extra Challenge), la normalización ayuda a estabilizar el entrenamiento al mantener valores dentro de un rango bien definido.
5. *Capa de salida (self.fc)*. Es una capa completamente conectada (*Linear(embed_dim, 1)*) que reduce la representación final a una salida para clasificación.
6. Propagación hacia adelante (forward)
	- *embedded = self.embedding(x)*: Convierte los tokens en vectores densos de embed_dim dimensiones. Salida (batch_size, seq_len, embed_dim).
	- *embedded = self.pos_encoding(embedded)*: Se suma la codificación posicional para agregar información sobre el orden de la secuencia.
	- *embedded = embedded.permute(1, 0, 2)*: Reorganiza las dimensiones para que coincida con la entrada esperada por nn.MultiheadAttention. Nueva forma (seq_len, batch_size, embed_dim). Esto es necesario porque PyTorch implementa nn.MultiheadAttention esperando el formato (longitud de secuencia, tamaño del lote, dimensiones de embedding).



In [30]:
# Task 2: Implement Multi-Head Attention (Replace Mean Pooling)
class AttentionModel(nn.Module):
    def __init__(self, embed_dim, num_heads, vocab_size, max_len=5000):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoding = PositionalEncoding(embed_dim, max_len)
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.norm = nn.LayerNorm(embed_dim)
        self.fc = nn.Linear(embed_dim, 1)

    def forward(self, x, attention_mask=None):
        embedded = self.embedding(x)  # Shape: (batch_size, seq_len, embed_dim)
        embedded = self.pos_encoding(embedded)  # Apply positional encoding
        embedded = embedded.permute(1, 0, 2)  # Change shape for MultiheadAttention (seq_len, batch_size, embed_dim)
                
# Task 3: Replace Mean Pooling with Multi-Head Attention

#En esta tarea, se reemplaza la estrategia de Mean Pooling por Atención Multi-Cabeza 
# para mejorar la capacidad del modelo de capturar relaciones contextuales dentro de 
# la secuencia. La línea attn_output, _ = 
# self.multihead_attn(embedded, embedded, embedded, key_padding_mask=attention_mask) 
# aplica la Autoatención Multi-Cabeza, donde cada palabra en la secuencia puede atender 
# a otras palabras, permitiendo que el modelo aprenda representaciones más ricas y 
# contextualmente informadas. Posteriormente, attn_output = attn_output.permute(1, 0, 2) 
# reorganiza las dimensiones para restaurar el formato esperado (batch_size, seq_len, embed_dim). 
# Finalmente, se aplica Normalización por Capas (self.norm(attn_output)) para estabilizar 
# el entrenamiento y mejorar la convergencia del modelo. Este enfoque optimiza la representación 
# de las secuencias, mejorando la capacidad del modelo para comprender la estructura del 
# lenguaje.

        attn_output, _ = self.multihead_attn(embedded, embedded, embedded, key_padding_mask=attention_mask)
        attn_output = attn_output.permute(1, 0, 2)  # Back to (batch_size, seq_len, embed_dim)
        attn_output = self.norm(attn_output)  # Layer normalization (Extra Challenge)
        
# Task 4: Extract First Token Representation (CLS Token Equivalent)

# En esta tarea, se extrae la representación del primer token de la secuencia 
# (CLS token equivalent), lo cual es una técnica común en modelos de procesamiento de 
# lenguaje natural para capturar una representación global del texto. 
# La operación pooled_output = attn_output[:, 0, :] selecciona únicamente la salida 
# correspondiente al primer token de la secuencia, que contiene información agregada a 
# partir del mecanismo de atención. Finalmente, esta representación pasa a través de 
# una capa totalmente conectada (self.fc(pooled_output)) y se aplica la función sigmoide 
# (torch.sigmoid) para obtener una probabilidad en una tarea de clasificación binaria. 
# Este enfoque permite que el modelo aprenda una representación compacta y efectiva del 
# texto de entrada.

        pooled_output = attn_output[:, 0, :]  # Take the first token output
        
        return torch.sigmoid(self.fc(pooled_output))  # Binary classification output


In [31]:
# Extra Challenges:
# 1. Experimenting with different pooling strategies

# En este desafío adicional, se implementa una variante del modelo que reemplaza la 
# estrategia de extracción del primer token (CLS token equivalent) por una técnica de 
# max pooling. En lugar de seleccionar únicamente la representación del primer token, 
# la operación torch.max(attn_output, dim=1) extrae el valor máximo a lo largo de la 
# dimensión de secuencia, lo que permite capturar la característica más relevante de 
# cada dimensión del espacio latente. Este enfoque puede ser útil en tareas donde la 
# información más importante no siempre se concentra en la primera posición de la secuencia. 
# Finalmente, la salida pasa por una capa totalmente conectada seguida de una función sigmoide
# para la clasificación binaria.
class MaxPoolingModel(AttentionModel):
    def forward(self, x, attention_mask=None):
        embedded = self.embedding(x)
        embedded = self.pos_encoding(embedded)
        embedded = embedded.permute(1, 0, 2)
        attn_output, _ = self.multihead_attn(embedded, embedded, embedded, key_padding_mask=attention_mask)
        attn_output = attn_output.permute(1, 0, 2)
        attn_output = self.norm(attn_output)
        pooled_output, _ = torch.max(attn_output, dim=1)  # Max pooling instead of CLS token
        return torch.sigmoid(self.fc(pooled_output))

# 2. Comparing learned positional embeddings vs. sinusoidal encodings

# n esta implementación, se compara el uso de codificaciones posicionales aprendidas con 
# las codificaciones sinusoidales predefinidas. A diferencia del método basado en funciones 
# trigonométricas, aquí se utiliza una matriz de parámetros entrenable (nn.Parameter), 
# lo que permite que el modelo aprenda una representación posicional óptima a partir de 
# los datos. Durante la propagación, la codificación aprendida se suma a las representaciones 
# de los tokens de entrada, similar a la versión sinusoidal. Posteriormente, se integra en 
# un modelo de Multi-Head Attention para evaluar su desempeño. Esta comparación es útil para 
# determinar si una codificación fija es suficiente o si el modelo se beneficia de aprender 
# sus propias posiciones.

class LearnedPositionalEncoding(nn.Module):
    def __init__(self, max_len, embed_dim):
        super().__init__()
        self.encoding = nn.Parameter(torch.randn(1, max_len, embed_dim))

    def forward(self, x):
        return x + self.encoding[:, :x.size(1)]  # Learned encoding instead of sinusoidal

# Example usage
if __name__ == "__main__":
    vocab_size = 10000
    embed_dim = 128
    num_heads = 8
    seq_len = 50
    batch_size = 32
    
    model = AttentionModel(embed_dim, num_heads, vocab_size)
    input_ids = torch.randint(0, vocab_size, (batch_size, seq_len))
    attention_mask = torch.zeros(batch_size, seq_len).bool()  # No padding in this case
    output = model(input_ids, attention_mask)
    print("Model Output Shape:", output.shape)


Model Output Shape: torch.Size([32, 1])


In [ ]:
# Valores de la codificación posicional aprendida
print("Learned Positional Encoding:", model.pos_encoding.encoding.shape)

Learned Positional Encoding: torch.Size([1, 5000, 128])


In [33]:
# Dimensiones de entrada y salida
print("Input Shape:", input_ids.shape)
print("Output Shape:", output.shape)

Input Shape: torch.Size([32, 50])
Output Shape: torch.Size([32, 1])


In [ ]:
# verificar cómo los tokens son transformados por la capa de embedding
embedded = model.embedding(input_ids)
print("Sample Embedding Shape:", embedded.shape)
print("Sample Embedding Values:", embedded[0, :5])  # Primeros 5 tokens de la primera muestra

Sample Embedding Shape: torch.Size([32, 50, 128])
Sample Embedding Values: tensor([[-6.3791e-01, -3.0183e-01,  1.0202e-01,  7.6067e-01, -8.1699e-01,
         -1.4822e+00,  4.2339e-01,  3.9453e-01, -2.6421e-01, -8.8591e-01,
         -5.4635e-01, -1.3147e+00, -2.7177e+00,  4.0065e-01,  5.7290e-01,
          3.9411e-01, -7.6752e-01,  5.8985e-01, -2.5876e-01,  1.3223e+00,
         -4.1940e-01,  5.9033e-01,  6.6007e-01,  1.0046e+00, -2.6067e-01,
         -3.6654e-01, -9.4022e-01,  1.6248e+00, -7.6404e-01,  5.7202e-01,
         -1.3360e+00, -5.2518e-01, -5.3944e-01,  8.3092e-01, -5.8592e-02,
         -2.5962e-01, -1.3398e+00,  1.0622e+00, -7.0613e-01,  1.1874e+00,
          8.2952e-01, -1.0658e+00,  2.4963e+00,  1.5158e+00, -7.7159e-01,
          7.5991e-01, -1.5570e+00, -7.7714e-01, -8.5059e-01,  8.0556e-01,
         -1.7259e+00,  4.3618e-01, -1.1123e+00, -8.0159e-02, -1.2766e+00,
         -1.9401e+00, -4.9817e-01,  9.2177e-01, -1.0612e+00,  1.6774e+00,
          7.3943e-01,  2.0938e+00, -4

In [ ]:
# salida de Multi-Head Attention
attn_output, _ = model.multihead_attn(embedded.permute(1, 0, 2), 
                                      embedded.permute(1, 0, 2), 
                                      embedded.permute(1, 0, 2))
print("Attention Output Shape:", attn_output.shape)
print("Sample Attention Output:", attn_output[:, 0, :5])  # Primer token de cada batch

Attention Output Shape: torch.Size([50, 32, 128])
Sample Attention Output: tensor([[-0.0665, -0.0585,  0.1253, -0.0762, -0.0421],
        [-0.0610, -0.0771,  0.0843,  0.0073, -0.0215],
        [ 0.0152, -0.0755,  0.0952, -0.0484, -0.0589],
        [-0.0676, -0.0711,  0.0633, -0.0788, -0.0433],
        [ 0.0078, -0.0638,  0.0884, -0.1008,  0.0103],
        [-0.0456, -0.0716,  0.0827, -0.0506, -0.0726],
        [ 0.0082, -0.0847,  0.0376, -0.0934, -0.0182],
        [ 0.0552, -0.0753,  0.0587, -0.0423, -0.0822],
        [-0.0323, -0.1294,  0.1168, -0.0376, -0.0133],
        [ 0.0132, -0.0809,  0.0042, -0.0133, -0.0760],
        [-0.0636, -0.1057,  0.0407, -0.0898, -0.0819],
        [-0.1566, -0.1415,  0.1026, -0.0461, -0.0417],
        [-0.0842, -0.1320,  0.0410, -0.0586, -0.1178],
        [-0.0373, -0.0883,  0.0274, -0.0494, -0.1076],
        [-0.0777, -0.1117,  0.0767, -0.0601, -0.0662],
        [ 0.0639, -0.1075,  0.1280, -0.0461,  0.0429],
        [-0.0687, -0.0609,  0.1540, -0.0870, 

In [ ]:
#pesos del clasificador final y ver su distribución
print("Final Layer Weights:", model.fc.weight.shape)
print("Final Layer Bias:", model.fc.bias)

Final Layer Weights: torch.Size([1, 128])
Final Layer Bias: Parameter containing:
tensor([-0.0249], requires_grad=True)


Los resultados obtenidos muestran la configuración de la capa final del modelo, la cual es una capa lineal de tamaño (1, 128), indicando que cada una de las 128 dimensiones del embedding contribuye a la salida del modelo mediante un peso específico. Estos pesos son ajustados durante el entrenamiento para optimizar la clasificación de los datos. Adicionalmente, el sesgo de la capa tiene un valor inicial de -0.0249, lo que sugiere un pequeño desplazamiento en la función de decisión antes de aplicar la activación sigmoide. Este sesgo permite mejorar la capacidad del modelo para generalizar al ajustar la frontera de decisión. Estos parámetros confirman que el modelo está correctamente estructurado para realizar clasificación binaria a partir de representaciones densas de las secuencias de entrada.